In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# To load the environment variable defined in the .env file
from dotenv import load_dotenv
load_dotenv();

In [4]:
import os

os.getcwd()

'C:\\Users\\anniw\\IFT6758-A2022-G08\\notebooks\\Milestone-2'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import src.visualization.visualize as VizManager
import src.features.build_features as FeaturesManager

ModuleNotFoundError: No module named 'src'

In [ ]:
SEED = 42

In [ ]:
seasons_year = [2015, 2016, 2017, 2018]
season_type = "Regular"
features_data = FeaturesManager.build_features(seasons_year, season_type)

In [ ]:
features_data.info()

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline # Outlier not integrated in pipeline actually

In [ ]:
# Separate features from target
X = features_data.drop(['Is Goal', 'Type'], axis=1)
y = features_data['Is Goal']

In [ ]:
# Data split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# Check how many shots are in defensive zone - these are potentially outliers if not empty net
out = X_train[X_train['st_X'] < -25]['st_X'].count()
X_train.shape[0] - out # we should expect approximately this much rows left after outlier removal

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Outliers - Boxplot
sns.boxplot(x=X_train['st_X']) # hahaha

In [ ]:
# Distribution before outliers removal
sns.histplot(X_train['st_X'])

In [ ]:
# Outliers - Percentile
min_thresold, max_thresold = X_train['st_X'].quantile([0.05, 0.95]) # define threshold

percentile_mask = ~(X_train['st_X']<min_thresold) | (X_train['st_X']>max_thresold) # create bool mask for threshold

X_train_percentile = X_train[percentile_mask]
y_train_percentile  = y_train[percentile_mask]

X_train_percentile.shape, y_train_percentile.shape

In [ ]:
sns.histplot(X_train_percentile['st_X'])

In [ ]:
# Outliers - IQR
Q1 = X_train['st_X'].quantile(0.25)
Q3 = X_train['st_X'].quantile(0.75)
IQR = Q3 - Q1

lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR

IQR_mask = ~(X_train['st_X']<lower_limit) | (X_train['st_X']>upper_limit)
X_train_IQR = X_train[IQR_mask]
y_train_IQR = y_train[IQR_mask]

X_train_IQR.shape, X_train_IQR.shape 

In [ ]:
sns.histplot(X_train_IQR['st_X'])

In [ ]:
# Outliers - STD
factor = 2
upper_limit = X_train['st_X'].mean() + factor*X_train['st_X'].std()
lower_limit = X_train['st_X'].mean() - factor*X_train['st_X'].std()

std_mask = (X_train['st_X']<upper_limit) & (X_train['st_X']>lower_limit)

X_train_std = X_train[std_mask]
y_train_std = y_train[std_mask]

X_train_std.shape, X_train_std.shape 

In [ ]:
sns.histplot(X_train_std['st_X'])

In [ ]:
# Outliers - Z-Score
threshold = 3
zscore_mask = ~(np.abs(stats.zscore(X_train['st_X'])) > threshold)

X_train_zscore = X_train[zscore_mask]
y_train_zscore = y_train[zscore_mask]

X_train_zscore.shape, y_train_zscore.shape

In [ ]:
sns.histplot(X_train_zscore['st_X'])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
test = pd.DataFrame(scaler.fit_transform(X_train[['st_X']]))

In [ ]:
sns.histplot(test[0])

In [ ]:
from sklearn.svm import OneClassSVM

clf = OneClassSVM(gamma='auto').fit_predict(X_train[['st_X']]) # So slow, i killed the kernel, not sure how to interpret the result

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor().fit_predict(X_train[['st_X']])